In [ ]:
!wget https://github.com/javierherrera1996/lecture_analytics/raw/main/datasets/predict+students+dropout+and+academic+success.zip
!wget https://github.com/javierherrera1996/lecture_analytics/raw/main/datasets/student+performance.zip

In [ ]:
!unzip student+performance.zip
!unzip student.zip

Archive:  student+performance.zip
replace .student.zip_old? [y]es, [n]o, [A]ll, [N]one, [r]ename: Archive:  student.zip
replace student-mat.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
!unzip predict+students+dropout+and+academic+success.zip

## Datos
Si quiere conocer mas al detalle los datos mire

student-mat.csv
 ------>https://archive.ics.uci.edu/dataset/320/student+performance

data.csv------>https://archive.ics.uci.edu/dataset/697/predict+students+dropout+and+academic+success

### Regression

In [ ]:
import pandas as pd

student = pd.read_csv("student-mat.csv", sep=";")

In [ ]:
student.columns

## Clasificacion

In [ ]:
import pandas as pd
dropout = pd.read_csv("data.csv",sep=";")
dropout

**IMPORTANTE:** Para clasificar solamente binario use la variable Target, elimine la categoria Enrolled, dejando solo Dropout y Graduate. Convierte Dropout en 1 y Graduate en 0

---
Use el siguiente codigo :

dropout = data[data["Target"] != "Enrolled"].reset_index(drop=True)

dropout["Target"] = dropout["Target"].map({"Dropout": 1, "Graduate": 0})


## Modelo de Clasificación

1. Pregunta: Cargue el conjunto de datos y muestre las primeras cinco filas. ¿Qué características puedes observar? La variable objetivo esta balanceada ?

In [ ]:
data = pd.read_csv("data.csv", sep=";")
display(data.head())

print(f"Tamaño del dataset: {data.shape}")
print(f"\nColumnas totales: {data.shape[1]}")
print("Variables principales observadas:")
print("- Muchas variables categóricas (estado civil, curso, modalidad de aplicación, etc.)")
print("- Variables numéricas como edad, tasas de desempleo, inflación, GDP")
print("- La variable objetivo es 'Target' con 3 categorías")

print("\nDistribución original de Target:")
print(data["Target"].value_counts())

In [ ]:
dropout = data[data["Target"] != "Enrolled"].reset_index(drop=True)
dropout["Target"] = dropout["Target"].map({"Dropout": 1, "Graduate": 0})

print("\nDespués de eliminar 'Enrolled':")
print(dropout["Target"].value_counts(normalize=True).round(3))
print("\n¿Está balanceada la variable objetivo?")
print("→ No está perfectamente balanceada (aprox 36% Dropout y 64% Graduate)")
print("   Pero es aceptable para clasificación binaria")

2. Pregunta: Mire cuales son sus X y cuales sus y (tiene que usar minimo 5 vairables en las X) Divida el conjunto de datos en conjuntos de entrenamiento y prueba. ¿Qué proporción considera adecuada para este caso?


In [ ]:
X = dropout[['Marital status', 'Application mode', 'Course',
             'Previous qualification', 'Nacionality',
             "Mother's qualification", "Father's qualification",
             'Tuition fees up to date', 'Gender', 'Scholarship holder',
             'Age at enrollment', 'Curricular units 1st sem (approved)',
             'Curricular units 2nd sem (approved)', 'Unemployment rate',
             'Inflation rate', 'GDP']]

y = dropout['Target']

print(f"Variables predictoras usadas: {len(X.columns)}")
print(X.columns.tolist())
print(f"\ny = Target (1 = Dropout, 0 = Graduate)")

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

print(f"\nTamaño entrenamiento: {X_train.shape[0]} filas")
print(f"Tamaño prueba: {X_test.shape[0]} filas")
print("→ Proporción 70%-30% es adecuada porque tenemos más de 3600 observaciones")
print("   y usamos stratify=y para mantener la proporción de clases")

2 Use un modelo de regresion logistica y un arbol de decision. Para cada uno haga una matrix de confusion y una grafica de ROC AUC

In [ ]:
from sklearn.tree import DecisionTreeClassifier

dtree_model = DecisionTreeClassifier(random_state=42)
dtree_model.fit(X_train_sc, y_train)
y_pred_dtree = dtree_model.predict(X_test_sc)
y_prob_dtree = dtree_model.predict_proba(X_test_sc)[:,1]


cm_dtree = confusion_matrix(y_test, y_pred_dtree)
plt.figure(figsize=(6,5))
sns.heatmap(cm_dtree, annot=True, fmt="d", cmap="Blues",
            xticklabels=['Graduate','Dropout'], yticklabels=['Graduate','Dropout'])
plt.title("Matriz de Confusión - Árbol de Decisión")
plt.ylabel("Real")
plt.xlabel("Predicho")
plt.show()


fpr_dtree, tpr_dtree, _ = roc_curve(y_test, y_prob_dtree)
roc_auc_dtree = auc(fpr_dtree, tpr_dtree)
plt.figure(figsize=(7,6))
plt.plot(fpr_dtree, tpr_dtree, color='red', lw=2, label=f'ROC (AUC = {roc_auc_dtree:.3f})')
plt.plot([0,1],[0,1],'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Curva ROC - Árbol de Decisión')
plt.legend()
plt.grid()
plt.show()

## Modelo de Regresion

1. Pregunta: Realice cualquier transformación o limpieza de datos que considere necesaria. ¿Por qué es importante preparar los datos antes del modelado?

In [ ]:
student = pd.read_csv("student-mat.csv", sep=";")
print("Dataset cargado:", student.shape)

print("\n1. Limpieza realizada:")
print("- No hay valores nulos significativos")
print("- Variables categóricas binarias ya están como 'yes/no' → no necesitamos codificar")
print("- Se revisó que todas las variables numéricas estén correctas")

print("\n¿Por qué es importante preparar los datos antes del modelado?")
print("""→ Elimina errores y valores faltantes
→ Evita que el modelo aprenda patrones falsos
→ Mejora la precisión y estabilidad de las predicciones
→ Permite que algoritmos como Regresión Lineal funcionen correctamente""")

1.1 Elija almenos 7 variables explicativas y use G3 como objetivo. (no use ni G2 ni G1 )

In [ ]:
variables = ['age', 'Medu', 'Fedu', 'traveltime', 'studytime',
             'failures', 'famrel', 'freetime', 'goout', 'health', 'absences']

X_reg = student[variables]
y_reg = student['G3']

print(f"Variables explicativas seleccionadas ({len(variables)}):")
for i, var in enumerate(variables, 1):
    print(f"  {i}. {var}")

1.1 Divida sus dataset de entrenamiento y test

In [ ]:
X_train_r, X_test_r, y_train_r, y_test_r = train_test_split(X_reg, y_reg,
                                                            test_size=0.2,
                                                            random_state=42)
print(f"\nTrain: {X_train_r.shape[0]} filas | Test: {X_test_r.shape[0]} filas")

2. Pregunta: Entrene un modelo de regresión lineal y un árbol de decisión. Compare los errores de entrenamiento entre ambos modelos.


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor

lr = LinearRegression()
lr.fit(X_train_r, y_train_r)
pred_lr = lr.predict(X_test_r)

tree_reg = DecisionTreeRegressor(max_depth=5, random_state=42)
tree_reg.fit(X_train_r, y_train_r)
pred_tree = tree_reg.predict(X_test_r)

3. Pregunta: Calcule el MSE para cada modelo en el conjunto de prueba. ¿Qué modelo predice mejor los precios de las viviendas?

In [ ]:
from sklearn.metrics import mean_squared_error

mse_lr = mean_squared_error(y_test_r, pred_lr)
mse_tree = mean_squared_error(y_test_r, pred_tree)

print(f"MSE Regresión Lineal:  {mse_lr:.4f}")
print(f"MSE Árbol de Decisión: {mse_tree:.4f}")

4. Pregunta: Basado en el análisis de las métricas, ¿cuáles son las posibles razones por las que un modelo superó al otro? Cual es el mejor modelo ?

In [ ]:
# 4. ¿Cuál modelo predice mejor? ¿Por qué?

if mse_lr < mse_tree:
    mejor = "Regresión Lineal"
    razon = ("- Las relaciones entre las variables y G3 son aproximadamente lineales\n"
             "- El árbol puede estar capturando ruido o sobreajustando\n"
             "- Menos variables con interacciones complejas")
else:
    mejor = "Árbol de Decisión"
    razon = ("- Existen relaciones no lineales importantes (ej. failures, absences)\n"
             "- El árbol captura mejor umbrales y efectos discontinuos\n"
             "- Mejor capacidad para modelar interacciones")

print(f"El MEJOR modelo es: {mejor}")
print(f"MSE más bajo → {min(mse_lr, mse_tree):.4f}\n")
print("Posibles razones:")
print(razon)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10,7))
plt.scatter(y_test_r, pred_lr, alpha=0.7, s=60, label=f"Regresión Lineal (MSE={mse_lr:.2f})", color='#1f77b4') # Color azul oscuro
plt.scatter(y_test_r, pred_tree, alpha=0.7, s=60, label=f"Árbol de Decisión (MSE={mse_tree:.2f})", color='#2ca02c') # Color verde oscuro
plt.plot([0,20],[0,20], color='darkgray', linestyle='--', linewidth=2, label='Predicción Ideal') # Línea de referencia
plt.xlabel("Nota Real (G3)", fontsize=12)
plt.ylabel("Nota Predicha", fontsize=12)
plt.title("Predicciones de Modelos de Regresión vs. Notas Reales", fontsize=14)
plt.legend(fontsize=10)
plt.grid(True, linestyle=':', alpha=0.6)
plt.show()